In [67]:
import pandas as pd
import math
import numpy as np

## Section 1: Example Dataset from Lecture Slides

In [68]:
df = pd.read_csv('Decision Tree Example Car Dataset.csv')

In [69]:
df.columns = [col.lower() for col in df.columns]
df

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


In [70]:
df.loc[0]

type       suv
color      red
tyre     white
doors        2
label        1
Name: 0, dtype: object

In [71]:
# num_case_dict contains the data showing the population for each label
# ie. we have 30 rows with class label1, 26 rows with label2 and 22 rows with label3
# num_case_dict = { "label1" : 30, "label2": 26, "label3": 22}

def gen_num_case_dict(df):
  # complete your code here
  count  = df['label'].value_counts()
  num_case_dict = count.to_dict()
  return num_case_dict

In [72]:
num_case_dict = gen_num_case_dict(df)
num_case_dict

{0: 9, 1: 5}

In [73]:
assert num_case_dict == {1: 5, 0: 9}

In [74]:
# input num_case_array and return the entropy value

def cal_entropy(num_case_dict):
  # complete your code here
  sum = 0
  entropy_sum =0
  
  for value in num_case_dict.values():
    sum += value
    
  for value in num_case_dict.values():
    entropy_sum -= (value/sum)*(np.log2(value/sum))
    
  return entropy_sum

In [75]:
cal_entropy( num_case_dict )

0.9402859586706311

In [76]:
assert np.allclose(cal_entropy( num_case_dict ), 0.9402859586706311)

In [77]:
# split by color red
df_red = df[ df['color'] == 'red']
display(df_red)

num_case_dict_red = gen_num_case_dict(df_red)

print(num_case_dict_red)

entropy_red = cal_entropy(num_case_dict_red)

print('entropy_red', entropy_red)

entropy_red_weighted = (len(df_red)/len(df)) * entropy_red
entropy_red_weighted

,type,color,tyre,doors,label
0,suv,red,white,2,1
4,car,red,black,2,1
7,minivan,red,black,4,0
10,suv,red,black,2,0


{1: 2, 0: 2}
entropy_red 1.0


0.2857142857142857

In [78]:
# calculate the entropy gain from a decision tree split using a particular feature on a particular df

def cal_entropy_split_gain(df, split_feature):
    # complete your code here
    count  = df[split_feature].value_counts()
    entropyS = cal_entropy(gen_num_case_dict(df))
    
    for num,key in enumerate(count.keys()):
        df_color = df[df[split_feature] == key]
        num_case_dict = gen_num_case_dict(df_color)
        entropy = cal_entropy(num_case_dict)
        
        entropyS -= (len(df_color)/len(df)) * entropy
    
    entropy_gain = entropyS  
    return entropy_gain

In [79]:
cal_entropy_split_gain(df, 'color')

0.02922256565895487

In [80]:
assert np.allclose( cal_entropy_split_gain(df, 'color'), 0.02922256565895487 )

In [81]:
cal_entropy_split_gain(df, 'type')

0.19996253177061118

In [82]:
assert np.allclose( cal_entropy_split_gain(df, 'type'), 0.19996253177061118 )

In [83]:
cal_entropy_split_gain(df, 'doors')

0.15183550136234164

In [84]:
assert np.allclose( cal_entropy_split_gain(df, 'doors'), 0.15183550136234159 )

In [85]:
cal_entropy_split_gain(df, 'tyre')

0.048127030408269544

In [86]:
assert np.allclose( cal_entropy_split_gain(df, 'tyre'), 0.04812703040826949 )

In [87]:
# Given the df and available features for conducting split, find the feature that produces maximum entropy gain.

def choose_best_split(df, available_feature_array):
  # complete your code here
  feature_with_max_entropy_gain = {feature:cal_entropy_split_gain(df, feature) for feature in available_feature_array if cal_entropy_split_gain(df, feature) !=0}
  if feature_with_max_entropy_gain == {}:
    return 0
  else:
    return max(feature_with_max_entropy_gain, key=feature_with_max_entropy_gain.get)

In [88]:
columns_list = df.columns.tolist()
columns_list.remove('label')
best_split_feature = choose_best_split(df, columns_list)
best_split_feature

'type'

In [89]:
assert best_split_feature == 'type'

In [90]:
# Generate a dictionary containing the split key and resulting df due to the split key and split feature.

def gen_feature_split_result(df, split_feature):
  # complete your code here
  count  = df[split_feature].value_counts()
  df_split_dict = {feature:df[ df[split_feature] == feature] for _,feature in enumerate(count.keys())}
  return df_split_dict

In [91]:
df_split_dict = gen_feature_split_result(df, best_split_feature)
df_split_dict

{'suv':    type  color   tyre  doors  label
 0   suv    red  white      2      1
 3   suv  green  white      4      1
 8   suv  green  black      4      0
 9   suv   blue  black      2      0
 10  suv    red  black      2      0
 12  suv  green  black      2      0,
 'car':    type  color   tyre  doors  label
 1   car  green  black      2      1
 2   car   blue  white      2      1
 4   car    red  black      2      1
 6   car  green  white      4      0
 11  car   blue  black      4      0,
 'minivan':        type  color   tyre  doors  label
 5   minivan   blue  white      4      0
 7   minivan    red  black      4      0
 13  minivan  green  white      4      0}

In [92]:
assert len(df_split_dict) == 3
assert len(df_split_dict['suv']) == 6
assert len(df_split_dict['minivan']) == 3
assert len(df_split_dict['car']) == 5

In [93]:
df_split_dict['suv']

,type,color,tyre,doors,label
0,suv,red,white,2,1
3,suv,green,white,4,1
8,suv,green,black,4,0
9,suv,blue,black,2,0
10,suv,red,black,2,0
12,suv,green,black,2,0


In [94]:
df_split_dict['minivan']

,type,color,tyre,doors,label
5,minivan,blue,white,4,0
7,minivan,red,black,4,0
13,minivan,green,white,4,0


In [95]:
df_split_dict['car']

,type,color,tyre,doors,label
1,car,green,black,2,1
2,car,blue,white,2,1
4,car,red,black,2,1
6,car,green,white,4,0
11,car,blue,black,4,0


## Section 2: Create Tree Structure

In [96]:
class Node:
  def __init__(self, df, parent, parent_edge, parent_split_feature, available_features):
    self.df = df
    self.parent = parent
    self.parent_edge = parent_edge
    self.children = {}
    self.split_feature = None
    self.available_features = available_features
    self.explored = False
    self.predicted_label = None
    if parent is not None:
      parent.split_feature = parent_split_feature

  def add_child(self, split_key, new_node):
    self.children[split_key] = new_node

In [97]:
class TreeGraph:
  def __init__(self, df, available_features):
    self.root = Node(df, None, None, None, available_features)
    self.nodes = []
    self.nodes.append(self.root)

  def add_node(self, df, parent_node, parent_edge, parent_split_feature, available_features):
    new_node = Node(df, parent_node, parent_edge, parent_split_feature, available_features)
    self.nodes.append(new_node)
    if parent_node is not None:
      parent_node.add_child(parent_edge, new_node )

  def gen_decision_tree(self):
    # Complete this code here
    # When you run this function it should generate the full decision tree by creating all the nodes for the tree
    for node in self.nodes:
      if node.explored == False:
        best_split_feature = choose_best_split(node.df, node.available_features)
      
        if best_split_feature != 0:
          node.explored = True
          current_available_features = node.available_features.copy()
          features = gen_feature_split_result(node.df, best_split_feature)
          
          for name, data in features.items():
            self.add_node(data, node, name, best_split_feature,current_available_features)

In [98]:
type(df.columns)

pandas.core.indexes.base.Index

In [99]:
type("svdvdv")

str

In [100]:
d = ['type', 'color', 'tyre', 'doors']
d.remove("type")
print(d)
print(type(d))

['color', 'tyre', 'doors']
<class 'list'>


In [101]:
columns_list = df.columns.tolist()
columns_list.remove('label')
g = TreeGraph(df, columns_list)
g.gen_decision_tree()
g.nodes

In [102]:
assert len(g.nodes) == 8

In [103]:
assert g.root.split_feature == 'type'

In [104]:
assert len(g.root.children) == 3

In [105]:
g.root.children.items()

dict_items([('suv', <__main__.Node object at 0x16e2db610>), ('car', <__main__.Node object at 0x13c4caa10>), ('minivan', <__main__.Node object at 0x16e2e7250>)])

In [106]:
for node in g.nodes:
  if node.split_feature == 'doors':
    test_case = node.df
    display(test_case)

assert len(test_case) == 5
assert np.array_equal( test_case['color'].to_numpy() , np.array(['green', 'blue', 'red', 'green', 'blue'], dtype=object) )

,type,color,tyre,doors,label
1,car,green,black,2,1
2,car,blue,white,2,1
4,car,red,black,2,1
6,car,green,white,4,0
11,car,blue,black,4,0


In [107]:
for node in g.nodes:
  if node.split_feature == 'tyre':
    test_case = node.df
    display(test_case)

assert len(test_case) == 6
assert np.array_equal( test_case['color'].to_numpy() , np.array(['red', 'green', 'green', 'blue', 'red', 'green'], dtype=object) )

,type,color,tyre,doors,label
0,suv,red,white,2,1
3,suv,green,white,4,1
8,suv,green,black,4,0
9,suv,blue,black,2,0
10,suv,red,black,2,0
12,suv,green,black,2,0


In [108]:
for node in g.nodes:
  if node.split_feature == 'type':
    test_case = node.df
    display(test_case)

assert len(test_case) == 14
assert np.array_equal( test_case['color'].to_numpy() , np.array(['red', 'green', 'blue', 'green', 'red', 'blue', 'green', 'red', 'green', 'blue', 'red', 'blue', 'green', 'green'], dtype=object) )

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


## Section 3: Do Inference to Make Predictions

In [109]:
# Now let's do inference with the tree we've generated

In [110]:
df

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


In [111]:
import copy

df_prediction = df.copy()
df_prediction['prediction'] = None

In [112]:
df_prediction

,type,color,tyre,doors,label,prediction
0,suv,red,white,2,1,None
1,car,green,black,2,1,None
2,car,blue,white,2,1,None
3,suv,green,white,4,1,None
4,car,red,black,2,1,None
5,minivan,blue,white,4,0,None
6,car,green,white,4,0,None
7,minivan,red,black,4,0,None
8,suv,green,black,4,0,None
9,suv,blue,black,2,0,None


In [113]:
g

In [114]:
for node in g.nodes:
    print(node.split_feature)

type
tyre
doors
None
None
None
None
None


In [115]:
# This function should fill in the predicted values using our decision tree into the df in the column 'prediction'

def generate_prediction(df_prediction, treeGraph):
  # Complete the code here
  for index, row in df_prediction.iterrows():
    current_node = treeGraph.root
    
    compare_df_prediction = df_prediction.drop(columns="prediction")
    # find the current node
    for node in treeGraph.nodes:
      if node.df.equals(compare_df_prediction):
        current_node = node
    
    # goto child
    for node in treeGraph.nodes:
      # find the child
      if node.parent == current_node:
        #find the condition
        if node.parent_edge == row[current_node.split_feature]:
          # move from parent to child
          current_node = node
    
    df_prediction.at[index,"prediction"] = current_node.df['label'].mode()[0]
    
  return df_prediction

In [116]:
prediction_result_df = generate_prediction(df_prediction,g)
prediction_result_df

,type,color,tyre,doors,label,prediction
0,suv,red,white,2,1,1
1,car,green,black,2,1,1
2,car,blue,white,2,1,1
3,suv,green,white,4,1,1
4,car,red,black,2,1,1
5,minivan,blue,white,4,0,0
6,car,green,white,4,0,0
7,minivan,red,black,4,0,0
8,suv,green,black,4,0,0
9,suv,blue,black,2,0,0


In [117]:
assert np.array_equal( prediction_result_df['prediction'].to_numpy(), np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object) )

In [118]:
def prediction_accuracy(prediction_result_df):
  correct = 0
  for index, row in prediction_result_df.iterrows():
    if row['label'] == row['prediction']:
      correct = correct + 1
  accuracy = correct / len(prediction_result_df)
  return accuracy

In [119]:
prediction_accuracy_result = prediction_accuracy(prediction_result_df)
prediction_accuracy_result

1.0

In [120]:
assert prediction_accuracy_result == 1

## Section 4: Test with Proper Dataset

In [121]:
# Get dataset from here: https://archive.ics.uci.edu/dataset/19/car+evaluation

# buying - buying price
# maintainance - price of the maintenance
# doors - number of doors
# person - capacity in terms of persons to carry
# luggage_boot - the size of luggage boot
# safety - estimated safety of the car
# label - evaulation level (unacceptable, acceptable, good, very good)

df_car = pd.read_csv('car.data', sep=',', header=None)

# Assign column names to the DataFrame
df_car.columns = ['buying', 'maintainance', 'doors', 'person', 'luggage_boot', 'safety', 'label' ]

df_car.head()

,buying,maintainance,doors,person,luggage_boot,safety,label
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [122]:
from sklearn.model_selection import train_test_split

df_car_train, df_car_test = train_test_split(df_car, test_size=0.2, random_state=42)

In [123]:
columns_list = df_car.columns.tolist()
columns_list.remove('label')
g_car = TreeGraph(df_car_train, columns_list)
g_car.gen_decision_tree()

In [124]:
len(g_car.nodes)

349

In [125]:
g_car

In [126]:
for node in g_car.nodes:
    print(node.split_feature)

safety
None
person
person
buying
maintainance
None
buying
buying
None
luggage_boot
maintainance
maintainance
maintainance
buying
buying
buying
buying
maintainance
maintainance
maintainance
maintainance
maintainance
maintainance
maintainance
maintainance
None
maintainance
doors
luggage_boot
None
luggage_boot
luggage_boot
luggage_boot
None
luggage_boot
None
luggage_boot
luggage_boot
None
luggage_boot
luggage_boot
luggage_boot
luggage_boot
luggage_boot
luggage_boot
None
luggage_boot
luggage_boot
doors
luggage_boot
luggage_boot
luggage_boot
luggage_boot
None
None
doors
luggage_boot
luggage_boot
luggage_boot
None
None
None
None
None
None
None
None
None
None
None
luggage_boot
doors
doors
doors
doors
None
doors
luggage_boot
luggage_boot
luggage_boot
doors
luggage_boot
doors
luggage_boot
doors
None
None
None
None
None
None
None
None
None
None
maintainance
doors
None
None
None
None
doors
None
None
None
None
doors
None
doors
None
None
None
doors
None
doors
None
None
None
None
None
doors
None
Non

In [127]:
df_car_prediction = df_car_test.copy()
df_car_prediction['prediction'] = None
df_car_prediction_result = generate_prediction(df_car_prediction, g_car)

In [128]:
df_car_prediction_result['prediction']

599     unacc
1201     good
628     unacc
1498      acc
1263    unacc
        ...  
100     unacc
274     unacc
1206    unacc
101     unacc
1084    unacc
Name: prediction, Length: 346, dtype: object

In [129]:
df_car_prediction_result

,buying,maintainance,doors,person,luggage_boot,safety,label,prediction
599,high,high,4,2,med,high,unacc,unacc
1201,med,low,2,4,med,med,acc,good
628,high,high,5more,2,big,med,unacc,unacc
1498,low,high,5more,4,med,med,acc,acc
1263,med,low,4,more,med,low,unacc,unacc
...,...,...,...,...,...,...,...,...
100,vhigh,vhigh,5more,more,small,med,unacc,unacc
274,vhigh,med,4,2,med,med,unacc,unacc
1206,med,low,2,more,small,low,unacc,unacc
101,vhigh,vhigh,5more,more,small,high,unacc,unacc


In [130]:
prediction_accuracy_result = prediction_accuracy(df_car_prediction_result)
prediction_accuracy_result

0.9248554913294798

In [131]:
assert prediction_accuracy_result > 0.9

In [132]:
assert len(df_car_prediction_result[df_car_prediction_result['prediction'].isna()]) == 0